In [1]:
import selenium 
import csv
import re
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options

import time

In [2]:
df = pd.read_csv("scraped_pubchem_data_final.csv")
df

,Original Molecule,Parsed Molecule,Link,Result Type,CAS,Deprecated CAS,Compound/Substance
0,1-2-PROPANEDIOL_DIACETATE,"1,2 PROPANEDIOL DIACETATE",https://pubchem.ncbi.nlm.nih.gov/compound/12198,RELEVANT,623-84-7,"134236-23-0, 1432741-27-9, 1432741-27-9, 13423...",COMPOUND
1,1-HEXADECANOL,1 HEXADECANOL,https://pubchem.ncbi.nlm.nih.gov/compound/2682,RELEVANT,"36653-82-4, 36311-34-9, 124-29-8","168679-13-8, 124-29-8, 55069-45-9, 8014-51-5, ...",COMPOUND
2,1-OCTADECANOL,1 OCTADECANOL,https://pubchem.ncbi.nlm.nih.gov/compound/8221,RELEVANT,"112-92-5, 68911-61-5, 26762-44-7","193766-48-2, 8014-37-7, 8032-19-7, 8032-21-1, ...",COMPOUND
3,2-3-(2-IODOPROPYLIDENEDIOXY)PROPANOL,"2,3-(2 IODOPROPYLIDENEDIOXY)PROPANOL",NaN,Not Found,NaN,NaN,NaN
4,2-OXOGLUTARIC_ACID,2 OXOGLUTARIC ACID,https://pubchem.ncbi.nlm.nih.gov/compound/51,FEATURED,"328-50-7, 34410-46-3, 17091-15-5",27175-99-1,COMPOUND
...,...,...,...,...,...,...,...
2900,ZIZYPHUS_JUJUBA,ZIZYPHUS JUJUBA,https://pubchem.ncbi.nlm.nih.gov/substance/482...,RELEVANT,NaN,NaN,SUBSTANCE
2901,ZOLEDRONIC_ACID,ZOLEDRONIC ACID,https://pubchem.ncbi.nlm.nih.gov/compound/68740,FEATURED,118072-93-8,NaN,COMPOUND
2902,ZOLMITRIPTAN,ZOLMITRIPTAN,https://pubchem.ncbi.nlm.nih.gov/compound/60857,FEATURED,139264-17-8,NaN,COMPOUND
2903,ZOLPIDEM,ZOLPIDEM,https://pubchem.ncbi.nlm.nih.gov/compound/5732,FEATURED,82626-48-0,NaN,COMPOUND


In [3]:
def setup_webdriver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def custom_title_case(s):
    # Find the first occurrence of an alphabetical character
    match = re.search(r'[a-zA-Z]', s)
    if match:
        # Capitalize the found character and lowercase the rest of the string
        first_letter_index = match.start()
        return s[:first_letter_index].lower() + s[first_letter_index].upper() + s[first_letter_index+1:].lower()
    else:
        # Return the lowercase string if no alphabetical character is found
        return s.lower()

In [9]:
from concurrent.futures import ThreadPoolExecutor, TimeoutError

def search_for_synonym(synonym_elements, comparisons):
    # Iterate through elements to find a match
    for element in synonym_elements:
        element_text = element.text.strip().lower()
        if element_text in comparisons:
            return True  # Found a match
    return False  # No match found

def synonym_checker(name, parsed, link, driver):
    synonym = 2  # Default to an error state

    if not pd.isna(link) and 'pubchem' in str(link):
        try:
            driver.get(link)
            WebDriverWait(driver, 3).until(
                EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'section#Depositor-Supplied-Synonyms a[data-action="content-link"]'))
            )
            synonym_elements = driver.find_elements(By.CSS_SELECTOR, 'section#Depositor-Supplied-Synonyms a[data-action="content-link"]')
    
            parsed_hyphenated = re.sub(r'(\d) (\w)', r'\1-\2', parsed)
            first_capitalised_h = custom_title_case(parsed_hyphenated)
            first_capitalised_n = custom_title_case(name)
            first_capitalised_p = custom_title_case(parsed)
            
            comparisons = {name.lower(), parsed.lower(), name.upper(), parsed.upper(), name.title(), parsed.title(),
                           parsed_hyphenated.lower(), parsed_hyphenated.upper(), parsed_hyphenated.title(),
                           first_capitalised_h.lower(), first_capitalised_n.lower(), first_capitalised_p.lower()}
            
            with ThreadPoolExecutor(max_workers=1) as executor:
                future = executor.submit(search_for_synonym, synonym_elements, comparisons)
                try:
                    synonym_found = future.result(timeout=30)  # Set timeout to 10 seconds
                    synonym = 0 if synonym_found else 1
                except TimeoutError:
                    print("Timed out waiting for synonym match.")
                    synonym = 2  # Or another value to indicate a timeout occurred
    
        except (NoSuchElementException, TimeoutException):
            synonym = 2
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            synonym = 2
    else:
        print(synonym)
        return synonym

    print(synonym)
    return synonym


def scan(df):
    synonym_flags = []
    driver = setup_webdriver()  # Initialize the WebDriver once

    try:
        for row in df.index:
            check = synonym_checker(df['Original Molecule'][row], df['Parsed Molecule'][row], df['Link'][row], driver)
            if check == 1:
                synonym_flags.append("NO")
            elif check == 0:
                synonym_flags.append("YES")
            else:
                synonym_flags.append("MISSING")

            print(f"remaining: {2904 - len(synonym_flags)}")
                
    finally:
        driver.quit()  # Make sure to quit the WebDriver

    return synonym_flags   

In [10]:
list = scan(df)

0
remaining: 2903
0
remaining: 2902
0
remaining: 2901
2
remaining: 2900
0
remaining: 2899
2
remaining: 2898
0
remaining: 2897
0
remaining: 2896
2
remaining: 2895
0
remaining: 2894
0
remaining: 2893


KeyboardInterrupt: 

In [ ]:
df['Synonym?'] = list

In [ ]:
df.tail(10)

In [ ]:
df

In [ ]:
final_df_csv = df.to_csv('scraped_pubchem_data_with_synonyms.csv', index = False)